# PockDock

This notebook demonstrates how a variety of different tools can be glued together into an efficient and flexible workflow using **Crossflow**.

The workflow downloads a protein-ligand complex form the PDB, runs fpocket, then docks the ligand back into the biggest pocket found. Then it calculates the error between the crystal structure coordinates of the ligand and those of each docking pose, before and after least-squares fitting.

The notebook requires you to have versions of **fpocket**, **autodock tools** and **autodock vina** installed on the worker node(s) of your dask cluster, which either:

 - is up and running and identifiable via the file "dask.dat" in the current directory (created when dask-scheduler is started with the `--scheduler_file` option)
 - will be a local cluster created right here, in which case scheduler_file=None

In [1]:
from crossflow import filehandling, tasks, clients
import sys
from urllib.request import urlretrieve
import numpy as np
import mdtraj as mdt

Create a crossflow client, connected to a pool of workers (see introductory notes):

In [2]:
scheduler_file = None
client = clients.Client(scheduler_file=scheduler_file)
client

AttributeError: 'Client' object has no attribute 'client'

Make the SubprocessTasks for **fpocket** and **Vina**, and FunctionTasks for other tasks:

In [3]:
# The fpocket task:
fpocket = tasks.SubprocessTask('fpocket -f x.pdb')
fpocket.set_inputs(['x.pdb'])
fpocket.set_outputs(['x_out/x_out.pdb'])

In [15]:
# The vina task:
vina = tasks.SubprocessTask('mac_vina --receptor r.pdbqt --ligand l.pdbqt --out out.pdbqt'
                                 ' --center_x {xc} --center_y {yc} --center_z {zc}'
                                 ' --size_x {sx} --size_y {sy} --size_z {sz} > dock.log')
vina.set_inputs(['r.pdbqt', 'l.pdbqt', 'xc', 'yc', 'zc', 'sx', 'sy', 'sz'])
vina.set_outputs(['out.pdbqt', 'dock.log'])

In [5]:
# AutoDock Tool based tasks to prepare receptor and ligand for docking:
prep_receptor = tasks.SubprocessTask('adt prepare_receptor4.py -r x.pdb -o x.pdbqt')
prep_receptor.set_inputs(['x.pdb'])
prep_receptor.set_outputs(['x.pdbqt'])

prep_ligand = tasks.SubprocessTask('adt prepare_ligand4.py -l x.pdb -o x.pdbqt')
prep_ligand.set_inputs(['x.pdb'])
prep_ligand.set_outputs(['x.pdbqt'])

In [6]:

def _download_and_split(pdb_code, ligand_residue_name):
    '''
    A function to download a pdb file, and split into receptor and ligand
    
    Args:
        pdb_code (str): 4-letter PDB code
        ligand_residue_name (str): 3-letter residue name
        
    Returns:
        mdt.trajectory: the receptor (protein atoms only)
        mdt.trajectory: the ligand
    '''
    pdb_file = pdb_code + '.pdb'
    path = urlretrieve('http://files.rcsb.org/download/' + pdb_file, pdb_file)
    hydrated_complex = mdt.load(pdb_file)
    receptor_atoms = hydrated_complex.topology.select('protein and chainid 0')
    found = False
    for chain in hydrated_complex.topology.chains:
        for r in chain.residues:
            if r.name == ligand_residue_name and not found:
                cid = chain.index
                found = True

    ligand_atoms = hydrated_complex.topology.select('resname {} and chainid {}'.format(ligand_residue_name, cid))
    receptor = mdt.load(pdb_file, atom_indices=receptor_atoms)
    ligand = mdt.load(pdb_file, atom_indices=ligand_atoms)
    return receptor, ligand
# Now make a FunctionTask for it:
download_and_split = tasks.FunctionTask(_download_and_split)
download_and_split.set_inputs(['pdb_code', 'ligand_residue_name'])
download_and_split.set_outputs(['receptor', 'ligand'])

In [7]:

def _pdbqt2pdb(infile):
    '''
    A Function to convert pdbqt files back to pdb ones
    
    Args:
        infile (str): name of the input file, .pdbqt format
    
    Returns:
        str: name of the .pdb file (always 'tmp.pdb')
    '''
    outfile = 'tmp.pdb'
    fout = open(outfile, 'w')
    with open(infile, 'r') as fin:
        for line in fin:
            if line[1:6] in 'ATOM  MODEL ENDMDL':
                fout.write(line)       
    fout.close()
    return 'tmp.pdb'

# Now make a FunctionTask for this:
pdbqt2pdb = tasks.FunctionTask(_pdbqt2pdb)
pdbqt2pdb.set_inputs(['infile'])
pdbqt2pdb.set_outputs(['outfile'])

In [8]:
def _get_dimensions(pockets):
    '''
    A Function to find the centre and extents of the largest pocket found by fpocket
    
    Args:
        pockets (str): Name of the pdb format file produced by fpocket
        
    Returns:
        (float,) * 6: the pocket centre and extents in x/y/z - in Angstroms
    '''
    buffer = 2.0
    t = mdt.load(pockets)
    site = t.topology.select('resname STP and residue 1') # This should be the largest pocket
    # In the next two lines, the factor of 10 is a conversion from nanometres to Angstroms:
    xc, yc, zc = tuple(10 * (t.xyz[0][site].min(axis=0) + t.xyz[0][site].max(axis=0)) / 2)
    sx, sy, sz = tuple(10 * (t.xyz[0][site].max(axis=0) - t.xyz[0][site].min(axis=0)) + buffer)
    return xc, yc, zc, sx, sy, sz

# Now make a FunctionTask for this:
get_dimensions = tasks.FunctionTask(_get_dimensions)
get_dimensions.set_inputs(['pockets'])
get_dimensions.set_outputs(['xc', 'yc', 'zc', 'sx', 'sy', 'sz'])

Now we construct the workflow. For convenience it's split up here into sections.

In [9]:
pdb_code = '1qy1'
ligand_residue_name = 'PRZ'

In [10]:
receptor, ligand, status = client.submit(download_and_split, pdb_code, ligand_residue_name)
print(status.result().returncode) # zero if successful

0


In [11]:
# Run fpocket:
pockets, status = client.submit(fpocket, receptor)
print(status.result().returncode)

0


In [12]:
# Find the dimensions of the biggest pocket
xc, yc, zc, sx, sy, sz, status = client.submit(get_dimensions, pockets)
print(status.result().returncode)

0


In [13]:
# Prepare receptor and ligand for docking:
receptor_qt, status = client.submit(prep_receptor, receptor)
print(status.result().returncode)
ligand_qt, status = client.submit(prep_ligand, ligand)
print(status.result().returncode)

0
0


In [16]:
# Run vina:
docks, logfile, status = client.submit(vina, receptor_qt, ligand_qt, xc, yc, zc, sx, sy, sz)
print(status.result().returncode)

0


In [17]:
# Check the log file:
print(logfile.result().read_text())

AutoDock Vina v1.2.3-9-g75f87a4-mod
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scr

In [31]:
# Convert the docked poses back to PDB format, and calculate unfitted and fitted rmsds 
# using MDTraj:
pdbout, status = client.submit(pdbqt2pdb, docks)
print(status.result().returncode)
docktraj = mdt.load(pdbout.result())

dxyz = docktraj.xyz - ligand.result().xyz
msd = (dxyz * dxyz).sum(axis=2).mean(axis=1)
unfitted_rmsd = np.sqrt(msd) * 10.0

rmsd = mdt.rmsd(docktraj, ligand.result()) * 10.0 # nm -> angstroms
print('Pose Fitted   Unfitted')
print('      rmsd      rmsd')
for mode in range(len(docktraj)):
    print('{:3d}   {:5.3f}    {:6.3f}'.format(mode+1, rmsd[mode], unfitted_rmsd[mode]))

0
Pose Fitted   Unfitted
      rmsd      rmsd
  1   1.209     2.836
  2   1.138     1.681
  3   1.073     3.553
  4   1.238     3.920
  5   0.304     4.580
  6   1.115     4.639
  7   1.197     3.611
  8   0.883     3.937
  9   0.351     4.274
